youtube 이수안 컴퓨터
XGBoost, LightGBM

In [18]:
import pandas as pd
import numpy as np
#from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, precision_score,recall_score

import xgboost as xgb
from xgboost import XGBRFClassifier,XGBRFRegressor
from xgboost import plot_importance, plot_tree

import graphviz
import matplotlib.pyplot as plt
plt.style.use(['seaborn-whitegrid'])


In [19]:
df=pd.read_csv('final_fire_weather_7yr.csv')

In [20]:
#변수지정
x = df[['평균기온', '일최고기온', '일최고기온시각', '일최저기온',
       '일최저기온시각', '일평균현지기압', '일평균해면기압', '일최고해면기압', '일최고해면기압시각', '일최저해면기압',
       '일최저해면기압시각', '일평균상대습도', '일최소상대습도', '일최소상대습도시각', '일평균수증기압', '일최고수증기압',
       '일최고수증기압시각', '일최저수증기압', '일최저수증기압시각', '일평균풍속', '일최대풍속', '일최대풍속시각',
       '일최대풍속풍향', '일최대순간풍속', '일최대순간풍속시각', '일최대순간풍속풍향', '풍정합', '일평균전운량',
       '일평균중하층운량', '최다운량', '최다운량시각', '일합계일조시간', '일합계일사량', '일평균지면온도']]  #변수 지정
y = df[['산불발생여부']]

In [21]:
X_train, X_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=123)
#파이썬 기반의 XGBoost가 가지고 있는 고유의 구조를 가지는 클래스를 가지고 있음
dtrain=xgb.DMatrix(data=X_train, label=y_train)
dtest=xgb.DMatrix(data=X_test, label=y_test)

In [24]:
params={
    'max_depth':3,
    'eta':0.1,
    'objective':'binary:logistic',
    'eval_metric':'logloss',
    'early_stopping':100
}
num_rounds=400

In [25]:
evals=[(dtrain,'train'),(dtest,'eval')]
xgb_model=xgb.train(params=params, dtrain=dtrain, num_boost_round=num_rounds, 
                    early_stopping_rounds=100, evals=evals)

[16:18:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stopping" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-logloss:0.64670	eval-logloss:0.64918
[1]	train-logloss:0.60903	eval-logloss:0.61343
[2]	train-logloss:0.57754	eval-logloss:0.58370
[3]	train-logloss:0.55109	eval-logloss:0.55878
[4]	train-logloss:0.52840	eval-logloss:0.53742
[5]	train-logloss:0.50949	eval-logloss:0.51919
[6]	train-logloss:0.49291	eval-logloss:0.50391
[7]	train-logloss:0.47868	eval-logloss:0.49120
[8]	train-logloss:0.46621	eval-logloss:0.48052
[9]	train-logloss:0.45521	eval-logloss:0.47023
[10]	train-logloss:0.44580	eval-logloss:0.46234
[11]	train-logloss:0.43757	eval-logloss:0.45577
[12]	train-

In [26]:
predicts=xgb_model.predict(dtest)
np.round(predicts[:10],3)

array([0.067, 0.879, 0.019, 0.266, 0.038, 0.868, 0.011, 0.787, 0.021,
       0.116], dtype=float32)

In [29]:
preds=[1 if x>0.5 else 0 for x in predicts]
preds[:10]

[0, 1, 0, 0, 0, 1, 0, 1, 0, 0]

In [31]:
print('정확도:{}'.format(accuracy_score(y_test,preds)))
print('정밀도:{}'.format(precision_score(y_test,preds)))
print('재현율:{}'.format(recall_score(y_test,preds)))

정확도:0.8374205267938238
정밀도:0.7228260869565217
재현율:0.5095785440613027
